In [122]:
import os
import datetime
import warnings
import polars as pl
import pandas as pd
import altair as alt

from src.najdi_rok import najdi_rok
from src.pocet_stran import pocet_stran
from src.bez_bordelu import bez_bordelu
from src.alt_friendly import alt_friendly
from src.hezke_jmeno import hezke_jmeno
from src.kristi_promin import kristi_promin

pl.Config(tbl_rows=100)
alt.data_transformers.disable_max_rows()
alt.themes.register('irozhlas', kristi_promin)
alt.themes.enable('irozhlas')
warnings.filterwarnings('ignore')

In [123]:
df = pl.read_parquet(os.path.join("data/cnb_sloupce","100.parquet"))
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","leader.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","008.parquet")), left_on="001", right_on="001", how="left")
df = df.to_pandas()
df = df[df["leader"].str[6].isin(["a", "t"])]
df = df[~df["leader"].str[7].isin(["b", "i", "s", " "])]
df = df[(df["008"].str[15:17] == "xr") & (df["008"].str[35:38] == "cze")]
df = pl.from_pandas(df)
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","020.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","022.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","245.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","300.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","655.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","700.parquet")), left_on="001", right_on="001", how="left")
df = df.explode("022_a").filter(pl.col("022_a").is_null())
df = df.with_columns(pl.col('008').map_elements(najdi_rok, return_dtype=int).alias('rok'))
df = df.with_columns(pl.col('300_a').map_elements(pocet_stran, return_dtype=int).alias('stran'))
df = df.with_columns(pl.col('245_a').map_elements(bez_bordelu, return_dtype=str))
df = df.explode('245_p').with_columns(pl.col('245_p').map_elements(bez_bordelu, return_dtype=str))
print(len(df))
df = df.filter(pl.col("rok") >= 1800)

df = df.filter((~pl.col("245_h").str.contains("grafika")) | pl.col("245_h").is_null()).unique(subset=["008","100_a","245_a","245_p"], keep="first")
print(len(df))

716107
705240


In [124]:
aut = pl.read_parquet(os.path.join("data","aut_vyber.parquet"))
cesi = aut.explode("370_c").filter(pl.col("370_c").str.contains("Česk")).explode("100_7").select(pl.col("100_7")).to_series().to_list()
len(cesi)

363993

In [125]:
vek = pl.read_parquet(os.path.join("data","narozeni-umrti-gender.parquet"))
df = df.join(vek, on="100_7", how="left").with_columns((pl.col("rok") - pl.col("narozeni")).alias("vek"))

In [884]:
df_700 = pl.read_parquet(os.path.join("data","podil_lidi_s_pauzou.parquet")).with_columns(pl.col("rok").map_elements(lambda x: x.year)).filter(pl.col("rok").is_between(1910,1990))

In [127]:
alt.Chart(alt_friendly(df_700)).mark_line().encode(alt.X("rok:T"), alt.Y("podil_cechu_ill_10"))

alt.Chart(...)

In [128]:
alt.Chart(alt_friendly(df_700)).mark_line().encode(alt.X("rok:T"), alt.Y("podil_cechu_trl_10"))

alt.Chart(...)

## Autorstvo s pauzou a končící autorstvo

In [882]:
df_100 = pl.read_parquet(os.path.join("data","podil_autorstva_s_pauzou.parquet")).with_columns(pl.col("rok").map_elements(lambda x: x.year)).filter(pl.col("rok").is_between(1910,1990)).with_columns(pl.col("podil_cechu_15").map_elements(lambda x: round(x, 3)))
df_koncici = pl.read_parquet(os.path.join('data','podily_konciciho_autorstva_cs.parquet'))

In [833]:
df_100.sort(by="podil_cechu_15",descending=True)

rok,podil_vsech_5,podil_cechu_5,podil_vsech_10,podil_cechu_10,podil_vsech_15,podil_cechu_15,podil_vsech_20,podil_cechu_20
i64,f64,f64,f64,f64,f64,f64,f64,f64
1971,0.134589,0.138058,0.077327,0.079512,0.051757,0.054,0.029274,0.02978
1972,0.135693,0.13884,0.078817,0.080849,0.051871,0.054,0.02634,0.026774
1973,0.131488,0.135512,0.076957,0.078934,0.049816,0.052,0.023767,0.02419
1970,0.130173,0.132936,0.072534,0.074913,0.048828,0.051,0.031153,0.03208
1974,0.129536,0.133994,0.073644,0.076126,0.047552,0.049,0.021606,0.022257
1904,0.147108,0.142471,0.069542,0.067181,0.049816,0.048,0.023404,0.022008
1905,0.142077,0.139837,0.075538,0.074555,0.047573,0.047,0.022179,0.021142
1969,0.126603,0.127617,0.07062,0.072544,0.045513,0.047,0.032372,0.033701
1903,0.141346,0.135014,0.065562,0.061814,0.04829,0.046,0.025379,0.023587


### Záhada roku 1905

In [842]:
rok04 = df.filter(pl.col("rok") == 1904).select(pl.col("100_7")).to_series().to_list()
roky06az18 = df.filter(pl.col("rok").is_between(1906,1918)).select(pl.col("100_7")).to_series().to_list()
roky18plus = df.filter(pl.col("rok") > 1900).select(pl.col("100_7")).to_series().to_list()
odmlceni_05 = [x for x in rok04 if x in roky18plus and x not in roky06az18]
odmlceni_05

['xx0216654',
 'js2014851443',
 'jk01010730',
 'jk01091634',
 'jn20001103129',
 'jk01030065',
 'jk01141018',
 'ola2013784979',
 'jk01062342',
 'jk01130592',
 'jk01062310',
 'jn20000604279',
 'jk01151488',
 'jk01030297',
 'ola2002111218',
 'jk01052841',
 'jo2012737452',
 'jn20000701504',
 'jx20031118013',
 'jk01020537',
 'jk01111334',
 'jk01071896',
 'jk01101110',
 'jk01111952',
 'jn20010523013',
 'js2015855279',
 'jk01111440',
 'jk01022185',
 'jk01082083',
 'jk01102823',
 'jk01111026',
 'jk01011212',
 'xx0302684',
 'jo2002146583',
 'jk01083057',
 'jk01151203',
 'jk01060317',
 'jk01080686',
 'nlk20000086930',
 'jk01050689',
 'jk01061544',
 'jk01100485',
 'jk01141304',
 'jk01070488',
 'js2014851443',
 'jk01062841',
 'jk01093004',
 'jk01060403',
 'jk01053259',
 'jk01011759',
 'jk01121694',
 'jx20070912007',
 'jk01072081',
 'jk01101643',
 'jk01093017',
 'jk01151904',
 'jn20000810001',
 'jk01142125',
 'jk01033367',
 'jo2002109710',
 'jk01110427',
 'jk01111440',
 'jk01111041',
 'xx0205184',


In [846]:
df.filter(pl.col('100_7').is_in(odmlceni_05)).group_by(['jmeno','100_7']).len().sort(by='len',descending=True)

jmeno,100_7,len
str,str,u32
"""Ladislav Klíma""","""jk01060531""",83
"""Josef Koblic""","""jk01060903""",62
"""Antonín Gindely""","""jk01032434""",52
"""Adolf Eckert""","""jk01030294""",32
"""František Cajthaml-Liberté""","""jk01020537""",28
"""Josef Burian""","""jk01020372""",24
"""František Cyril Kampelík""","""jk01052820""",24
"""František Vlasák""","""jk01150047""",24
"""Martin von Cochem""","""jo2002109710""",22


In [848]:
', '.join(df.filter(pl.col('100_7').is_in(odmlceni_05)).group_by(['jmeno','100_7']).len().sort(by='len',descending=True).select(pl.col('jmeno')).to_series().to_list()[0:30])

'Ladislav Klíma, Josef Koblic, Antonín Gindely, Adolf Eckert, František Cajthaml-Liberté, František Vlasák, Josef Burian, František Cyril Kampelík, Martin von Cochem, Antonín Konstantin Viták, Hendrik Conscience, Quintus Flaccus Horatius, Jan Karník, Matyáš Blažek, Josef Kousal, František Ladislav Menhard, Leopold M. Zeithammer, Josef Barák, Jan Roubal, Jan Eckert, Antonín Sucharda, Eduard Georgievič Valečka, Eugen Kumičić, Prokop Zaletěl, Jan Pravoslav Koubek, František Karel Bakule, Václav Vondrák, Adolf L. Seidl, Otakar Hanuš, Václav Šílený'

In [886]:
graf_pauzy = alt.Chart(
    alt_friendly(df_100), width=300,
    title=alt.Title("…a jak velká se odmlčela na 15 a více let")
).mark_area(color='#E09DA3'
).encode(alt.X(
    "rok:T", title=None, axis=alt.Axis(domainOpacity=0, tickColor='#DCDDD6')
), alt.Y(
    "podil_cechu_15:Q", title=None, 
    axis=alt.Axis(domainOpacity=0, tickColor='#DCDDD6', labelExpr="datum.label * 100 + ' %'", orient='right', tickCount=3),
    scale=alt.Scale(domain=[0,0.4])
))
                    
rule1 = alt.Chart(alt.Data(values=[{'rok': '1948-02-25'}])).mark_rule(
    color='#81A9D5',
    strokeDash=[4, 4]  # Optional: makes the line dashed
).encode(
    x='rok:T'
)

rule2 = alt.Chart(alt.Data(values=[{'rok': '1968-08-21'}])).mark_rule(
    color='#81A9D5',
    strokeDash=[4, 4]  # Optional: makes the line dashed
).encode(
    x='rok:T'
)

# Add text annotation
text1 = alt.Chart(alt.Data(values=[{'rok': '1948-02-25', 'y': 0.3}])).mark_text(
    # angle=270,  # Rotates text vertically
    align='right',
    baseline='middle',
    dy=15,# Slight horizontal offset from the line
    dx=-8,
    text=['komunistický','převrat'],
    font='Asap'
).encode(
    x='rok:T',
    y=alt.value(0)  # Places text at bottom of chart
)


# Add text annotation
text2 = alt.Chart(alt.Data(values=[{'rok': '1968-08-21', 'y': 0.3}])).mark_text(
    # angle=270,  # Rotates text vertically
    align='left',
    baseline='middle',
    dy=15,# Slight horizontal offset from the line
    dx=8,
    text=['srpnová','okupace'],
    font='Asap'
).encode(
    x='rok:T',
    y=alt.value(0)  # Places text at bottom of chart
)

graf_pauzy_graf_komb = graf_pauzy + rule1 + text1 + rule2 + text2
graf_pauzy_graf_komb

alt.LayerChart(...)

In [874]:
df_koncici.sort(by="podil",descending=True)

rok,skoncilo,celkem,podil
i64,u32,u32,f64
2025,45,45,1.0
2024,2232,2252,0.991119
2023,1849,2780,0.665108
2022,1493,3140,0.475478
1805,2,5,0.4
1834,4,10,0.4
2021,1223,3296,0.371056
2020,1059,3329,0.318114
2019,929,3490,0.266189


In [888]:
graf_konce = alt.Chart(
    alt_friendly(df_koncici.filter(pl.col("rok").is_between(1900,2000))
                ), width=300,
                 title=alt.Title(["Jak velká část žijících českých","autorů zcela přestala publikovat…"])
).mark_area(color='#E09DA3'
).encode(
    alt.X("rok:T", title=None, axis=alt.Axis(domainOpacity=0, tickColor='#DCDDD6')),
    alt.Y("podil:Q", title=None, axis=alt.Axis(domainOpacity=0, tickColor='#DCDDD6', tickCount=3, labelExpr="datum.label * 100 + ' %'", orient='right')))
graf_konce

rule1 = alt.Chart(alt.Data(values=[{'rok': '1948-02-25'}])).mark_rule(
    color='#81A9D5',
    strokeDash=[4, 4]  # Optional: makes the line dashed
).encode(
    x='rok:T'
)

rule2 = alt.Chart(alt.Data(values=[{'rok': '1989-11-17'}])).mark_rule(
    color='#81A9D5',
    strokeDash=[4, 4]  # Optional: makes the line dashed
).encode(
    x='rok:T'
)

# Add text annotation
text1 = alt.Chart(alt.Data(values=[{'rok': '1948-02-25', 'y': 0.3}])).mark_text(
    # angle=270,  # Rotates text vertically
    align='right',
    baseline='middle',
    dy=15,# Slight horizontal offset from the line
    dx=-8,
    text=['komunistický','převrat'],
    font='Asap'
).encode(
    x='rok:T',
    y=alt.value(0)  # Places text at bottom of chart
)


# Add text annotation
text2 = alt.Chart(alt.Data(values=[{'rok': '1989-11-17', 'y': 0.3}])).mark_text(
    # angle=270,  # Rotates text vertically
    align='right',
    baseline='middle',
    dy=15,# Slight horizontal offset from the line
    dx=-8,
    text=['sametová','revoluce'],
    font='Asap'
).encode(
    x='rok:T',
    y=alt.value(0)  # Places text at bottom of chart
)

graf_konce_komb = graf_konce + rule1 + rule2 + text1 + text2

graf_konce_komb

alt.LayerChart(...)

In [890]:
alt.vconcat(graf_konce_komb, graf_pauzy_graf_komb).configure_view(stroke='transparent').resolve_scale(y='shared').configure_axisX(grid=False, domain=False)

alt.VConcatChart(...)

In [868]:
df_koncici

rok,skoncilo,celkem,podil
i64,u32,u32,f64
1805,2,5,0.4
1807,1,7,0.142857
1810,1,4,0.25
1813,1,6,0.166667
1814,1,9,0.111111
1815,2,10,0.2
1816,1,10,0.1
1818,1,10,0.1
1821,2,21,0.095238


In [866]:
alt.Chart(alt_friendly(df_koncici.filter(pl.col("rok").is_between(1900,2000))), width=300).mark_line().encode(alt.X("rok:T"), alt.Y("podil_ill:Q"))

alt.Chart(...)

In [136]:
alt.Chart(alt_friendly(df_koncici.filter(pl.col("rok").is_between(1900,2000))), width=300).mark_line().encode(alt.X("rok:T"), alt.Y("podil_trl:Q"))

alt.Chart(...)

In [137]:
df_koncici_autorstvo = pl.read_parquet(os.path.join('data','podily_konciciho_autorstva_cs.parquet'))
df_koncici_autorstvo.columns

['rok', 'skoncilo', 'celkem', 'podil']

In [852]:
smytec = alt.Chart(
    alt_friendly(df_koncici_autorstvo.filter(pl.col("rok").is_between(1850,2000))), width=300, title=alt.Title(["Jak velká část českých autorů vyšla naposledy…"])
).mark_area().encode(
    alt.X("rok:T", title=None, axis=alt.Axis(domainOpacity=0, tickColor='#DCDDD6')), 
    alt.Y("podil:Q", title=None, axis=alt.Axis(orient='right', domainOpacity=0, tickColor='#DCDDD6', labelExpr="datum.label * 100 + ' %'"))
)

pauza15 = alt.Chart(
    alt_friendly(df_100), width=300, title=alt.Title('…a jak velká část se odmlčela na 15 a více let')
).mark_area().encode(
    alt.X("rok:T", title=None, axis=alt.Axis(domainOpacity=0, tickColor='#DCDDD6')), 
    alt.Y("podil_cechu_15", title=None, axis=alt.Axis(orient='right', domainOpacity=0, tickColor='#DCDDD6', labelExpr="datum.label * 100 + ' %'"), 
          scale=alt.Scale(domain=[0,0.2]))
)

alt.vconcat(smytec, pauza15).configure_view(stroke='transparent').resolve_scale(color='independent',x="shared",y='shared')

alt.VConcatChart(...)

## Kdo zmizel

In [140]:
pred_unorem = set(df.filter(pl.col("rok") <= 1948).select(pl.col("100_7")).to_series().to_list())
po_revoluci = set(df.filter(pl.col("rok") >= 1988).select(pl.col("100_7")).to_series().to_list())
v_sedesatkach = set(df.filter(pl.col("rok").is_between(1960,1969)).select(pl.col("100_7")).to_series().to_list())
behem_socialismu = set(df.filter(pl.col("rok").is_between(1949,1987)).select(pl.col("100_7")).to_series().to_list())
behem_padesatek = set(df.filter(pl.col("rok").is_between(1949,1959)).select(pl.col("100_7")).to_series().to_list())
pred_normalizaci = set(df.filter(pl.col("rok").is_between(1966,1970)).select(pl.col("100_7")).to_series().to_list())
po_normalizaci = set(df.filter(pl.col("rok").is_between(1990,1994)).select(pl.col("100_7")).to_series().to_list())
behem_normalizace = set(df.filter(pl.col("rok").is_between(1972,1988)).select(pl.col("100_7")).to_series().to_list())

znormalizovani = pred_normalizaci.intersection(po_normalizaci).difference(behem_normalizace)
zunorovani = pred_unorem.intersection(po_revoluci).difference(behem_socialismu)

In [141]:
len(znormalizovani)

533

In [142]:
len(df.filter(pl.col("100_7").is_in(set(cesi).intersection(znormalizovani))).unique(subset=['100_7']))

419

In [143]:
df = df.with_columns(pl.col("100_a").map_elements(hezke_jmeno).alias("jmeno"))

In [218]:
df.columns

['100_ind1',
 '100_a',
 '100_7',
 '100_4',
 '100_d',
 '100_q',
 '100_c',
 '100_b',
 '100_e',
 '001',
 'leader',
 '008',
 '020_q',
 '020_c',
 '020_a',
 '020_z',
 '022_a',
 '022_y',
 '022_z',
 '022_ind1',
 '022_l',
 '245_ind1',
 '245_ind2',
 '245_a',
 '245_b',
 '245_c',
 '245_n',
 '245_p',
 '245_h',
 '245_f',
 '245_s',
 '300_a',
 '300_b',
 '300_c',
 '300_e',
 '300_f',
 '300_3',
 '655_ind2',
 '655_a',
 '655_7',
 '655_2',
 '655_ind1',
 '655_x',
 '655_z',
 '655_y',
 '700_ind1',
 '700_a',
 '700_4',
 '700_d',
 '700_7',
 '700_t',
 '700_q',
 '700_l',
 '700_ind2',
 '700_c',
 '700_b',
 '700_i',
 '700_m',
 '700_n',
 '700_k',
 '700_r',
 '700_p',
 '700_o',
 '700_s',
 '700_j',
 '700_6',
 '700_x',
 '700_e',
 '700_f',
 '700_5',
 '700_g',
 'rok',
 'stran',
 'narozeni',
 'umrti',
 'gender',
 'vek',
 'jmeno']

In [735]:
kolik = 25
znormalizovani_top_domaci = df.filter((pl.col('umrti') >= 1969) | (pl.col('umrti').is_null())).filter(pl.col("100_7").is_in(cesi)).filter(pl.col("rok").is_between(1965,1994)).filter(pl.col("100_7").is_in(znormalizovani)).group_by(["jmeno",'100_7']).len().sort(by='len',descending=True).head(kolik).select(pl.col("100_7")).to_series().to_list()
znormalizovani_top_domaci_razeni = df.filter((pl.col('umrti') >= 1969) | (pl.col('umrti').is_null())).filter(pl.col("100_7").is_in(cesi)).filter(pl.col("rok").is_between(1965,1994)).filter(pl.col("100_7").is_in(znormalizovani)).group_by(["jmeno",'100_7']).len().sort(by='len',descending=True).head(kolik).select(pl.col("jmeno")).to_series().to_list()
znormalizovani_top_zahranicni = df.filter((pl.col('umrti') >= 1969) | (pl.col('umrti').is_null())).filter(~pl.col("100_7").is_in(cesi)).filter(pl.col("rok").is_between(1965,1994)).filter(pl.col("100_7").is_in(znormalizovani)).group_by(["jmeno",'100_7']).len().sort(by='len',descending=True).head(kolik).select(pl.col("100_7")).to_series().to_list()
znormalizovani_top_zahranicni_razeni = df.filter((pl.col('umrti') >= 1969) | (pl.col('umrti').is_null())).filter(~pl.col("100_7").is_in(cesi)).filter(pl.col("rok").is_between(1965,1994)).filter(pl.col("100_7").is_in(znormalizovani)).group_by(["jmeno",'100_7']).len().sort(by='len',descending=True).head(kolik).select(pl.col("jmeno")).to_series().to_list()
znormalizovani_top_domaci_razeni

['Josef Škvorecký',
 'Ivan Klíma',
 'Václav Havel',
 'Karel Kaplan',
 'Anna Sedlmayerová',
 'Vladimír Škutina',
 'Jan Vladislav',
 'Karel Šiktanc',
 'Václav Černý',
 'Ivan Diviš',
 'Jan Procházka',
 'Milan Kundera',
 'Eva Kantůrková',
 'Jan Patočka',
 'Václav Renč',
 'Arnošt Lustig',
 'Karel Pecka',
 'Adolf Melezinek',
 'Ludvík Vaculík',
 'Zdeněk Kalista',
 'Stanislav Sohr',
 'František Kautman',
 'Václav Klaus',
 'Jena Javůrková',
 'Karel Mácha']

In [146]:
df.filter(pl.col("100_7").is_in(cesi)).filter(pl.col("rok").is_between(1966,1994)).filter(pl.col("100_7").is_in(znormalizovani)).group_by(["jmeno",'100_7']).agg(pl.col("stran").sum()).sort(by='stran',descending=True)

jmeno,100_7,stran
str,str,i64
"""Josef Škvorecký""","""jk01130413""",9738
"""Václav Havel""","""jk01040240""",5192
"""Karel Kaplan""","""jk01052903""",4970
"""Václav Černý""","""jk01021636""",4653
"""Eva Kantůrková""","""jk01052883""",4506
"""Arnošt Lustig""","""jk01072683""",4495
"""Felix Háj""","""jk01033019""",4469
"""Anna Sedlmayerová""","""jk01110583""",4294
"""Jan Procházka""","""jk01101145""",3696


In [147]:
# kolik = 20
# znormalizovani_top_domaci = df.filter(pl.col("100_7").is_in(cesi)).filter(pl.col("rok").is_between(1966,1994)).filter(pl.col("100_7").is_in(znormalizovani)).group_by(["jmeno",'100_7']).agg(pl.col("stran").sum()).sort(by='stran',descending=True).head(kolik).select(pl.col("100_7")).to_series().to_list()
# znormalizovani_top_domaci_razeni = df.filter(pl.col("100_7").is_in(cesi)).filter(pl.col("rok").is_between(1966,1994)).filter(pl.col("100_7").is_in(znormalizovani)).group_by(["jmeno",'100_7']).agg(pl.col("stran").sum()).sort(by='stran',descending=True).head(kolik).select(pl.col("jmeno")).to_series().to_list()
# znormalizovani_top_zahranicni = df.filter(~pl.col("100_7").is_in(cesi)).filter(pl.col("rok").is_between(1966,1994)).filter(pl.col("100_7").is_in(znormalizovani)).group_by(["jmeno",'100_7']).agg(pl.col("stran").sum()).sort(by='stran',descending=True).head(kolik).select(pl.col("100_7")).to_series().to_list()
# znormalizovani_top_zahranicni_razeni = df.filter(~pl.col("100_7").is_in(cesi)).filter(pl.col("rok").is_between(1966,1994)).filter(pl.col("100_7").is_in(znormalizovani)).group_by(["jmeno",'100_7']).agg(pl.col("stran").sum()).sort(by='stran',descending=True).head(kolik).select(pl.col("jmeno")).to_series().to_list()

In [254]:
znormalizovani_top_domaci

['jk01130413',
 'jk01040240',
 'jk01060513',
 'jk01130395',
 'jk01052903',
 'jk01110583',
 'jk01150006',
 'jk01021636',
 'jk01122896',
 'jk01091914',
 'jk01052883',
 'jk01101145',
 'jk01092241',
 'jk01022321',
 'jk01072683',
 'jn19990218045',
 'jk01102340',
 'jk01053268',
 'jk01052690',
 'jk01120290',
 'jk01081112',
 'jk01141068',
 'xx0020417',
 'jk01070894',
 'jk01062995']

In [274]:
znormalizovani_top_domaci_razeni

['Josef Škvorecký',
 'Václav Havel',
 'Ivan Klíma',
 'Anna Sedlmayerová',
 'Karel Kaplan',
 'Vladimír Škutina',
 'Jan Vladislav',
 'Václav Černý',
 'Karel Šiktanc',
 'Jan Procházka',
 'Ivan Diviš',
 'Milan Kundera',
 'Eva Kantůrková',
 'Jan Patočka',
 'Václav Renč',
 'Adolf Melezinek',
 'Arnošt Lustig',
 'Karel Pecka',
 'Stanislav Sohr',
 'Zdeněk Kalista',
 'František Kautman',
 'Václav Klaus',
 'Ludvík Vaculík',
 'Karel Mácha',
 'Jena Javůrková']

Načteme data vygenerovaná sešitem 174_zahranicni_vydani.ipynb.

In [725]:
dografu2 = pl.read_parquet(os.path.join('data','zahranicni_knihy_10_zakazanych_autoru.parquet')).select(pl.col(['jmeno','titul','rok'])).with_columns(pl.lit('zahranici').alias('kde')).filter(pl.col("rok").is_between(1930,2000)).filter(pl.col("jmeno").is_in(znormalizovani_top_domaci_razeni))

In [737]:
dografu1 = df.filter(pl.col("rok").is_between(1930,2000)).filter(pl.col("100_7").is_in(znormalizovani_top_domaci)).select(pl.col(['jmeno','245_a','rok'])).rename({'245_a':'titul'}).with_columns(pl.lit('domaci').alias('kde'))

In [739]:
dografu2

jmeno,titul,rok,kde
str,str,i64,str
"""Karel Kaplan""","""Die politischen Prozesse in de…",1986,"""zahranici"""
"""Karel Kaplan""","""Poválečné Československo :""",1985,"""zahranici"""
"""Karel Kaplan""","""Staat und Kirche in der Tschec…",1990,"""zahranici"""
"""Karel Kaplan""","""Das verhängnisvolle Bündnis :""",1984,"""zahranici"""
"""Milan Kundera""","""Amori ridicoli :""",1973,"""zahranici"""
"""Milan Kundera""","""A lét elviselhetetlen könnyűsé…",1994,"""zahranici"""
"""Milan Kundera""","""Avskedvalsen /""",1978,"""zahranici"""
"""Milan Kundera""","""Avskjedsvalsen :""",1978,"""zahranici"""
"""Milan Kundera""","""Das Buch den lächerlichen Lieb…",1987,"""zahranici"""


In [741]:
dografu1

jmeno,titul,rok,kde
str,str,i64,str
"""Karel Mácha""","""Revoluční humanismus St.K. Neu…",1954,"""domaci"""
"""Stanislav Sohr""","""Breviář""",1995,"""domaci"""
"""Eva Kantůrková""","""Památník""",1995,"""domaci"""
"""Arnošt Lustig""","""Hořká vůně mandlí""",1968,"""domaci"""
"""Václav Havel""","""Protokoly""",1966,"""domaci"""
"""Jan Procházka""","""Rok života""",1956,"""domaci"""
"""Anna Sedlmayerová""","""Čtyři detektivky""",1970,"""domaci"""
"""Vladimír Škutina""","""Rodič v síti""",1991,"""domaci"""
"""Josef Škvorecký""","""Podivný pán z Providence a jin…",1999,"""domaci"""


In [799]:
dografu = pl.concat([dografu1, dografu2])

dografu = dografu.join(
    df.filter(
        pl.col("rok").is_between(1930,2000)
    ).filter(
        pl.col("100_7").is_in(znormalizovani_top_domaci)
    ).group_by('jmeno').agg(pl.col('umrti').max()), on='jmeno',how='left'
)

dografu = dografu.with_columns(pl.when(pl.col('umrti') >= 2001).then(None).otherwise(pl.col('umrti')).alias("umrti")).with_columns(pl.col('umrti').map_elements(lambda x: datetime.date(year=int(x), month=1, day=1), return_dtype=pl.Date).cast(pl.Datetime))

In [801]:
dografu

jmeno,titul,rok,kde,umrti
str,str,i64,str,datetime[μs]
"""Karel Mácha""","""Revoluční humanismus St.K. Neu…",1954,"""domaci""",null
"""Stanislav Sohr""","""Breviář""",1995,"""domaci""",2000-01-01 00:00:00
"""Eva Kantůrková""","""Památník""",1995,"""domaci""",null
"""Arnošt Lustig""","""Hořká vůně mandlí""",1968,"""domaci""",null
"""Václav Havel""","""Protokoly""",1966,"""domaci""",null
"""Jan Procházka""","""Rok života""",1956,"""domaci""",1971-01-01 00:00:00
"""Anna Sedlmayerová""","""Čtyři detektivky""",1970,"""domaci""",1995-01-01 00:00:00
"""Vladimír Škutina""","""Rodič v síti""",1991,"""domaci""",1995-01-01 00:00:00
"""Josef Škvorecký""","""Podivný pán z Providence a jin…",1999,"""domaci""",null


In [312]:
norm_dom = alt.Chart(alt_friendly(dografu), title=alt.TitleParams(["Normalizační průrva českou literaturou"], subtitle=["Autoři a autorky žijící v době srpnové okupace, kterým toho vyšlo nejvíc","v druhé půlce 60. let a první půlce 90. let a zároveň nic mezi lety 1973 a 1987.","Žluté tečky jsou knihy vydané během normalizace v zahraničí."]), width=300).mark_circle() \
        .encode(
            x=alt.X("rok:T", title=None, axis=alt.Axis(domainOpacity=0, tickColor='#DCDDD6')), 
            y=alt.Y("jmeno:N", sort=znormalizovani_top_domaci_razeni, title=None, axis=alt.Axis(orient='left', domainOpacity=0, tickColor='#DCDDD6')), 
            yOffset=alt.YOffset("jitter:Q", scale=alt.Scale(range=[3, 15])),
            size=alt.condition(
                alt.datum.kde == 'domaci',
                alt.value(8),  # Size for domestic authors
                alt.value(5)   # Half size for foreign authors
            ),
            color=alt.condition(
                alt.datum.kde == 'domaci',
                alt.Color('domaci:N', 
                         scale=alt.Scale(range=['#D6534B', '#445B78', '#DB842F', '#70871E']), 
                         sort=znormalizovani_top_domaci_razeni,
                         legend=None),
                alt.value('#EED801') # '#81A9D5')  # Gray color for zahranici
            )).transform_calculate(jitter="sqrt(-2*log(random()))*cos(2*PI*random())")

# norm_zahr = alt.Chart(alt_friendly(df.filter(pl.col("rok").is_between(1930,2000)).filter(pl.col("100_7").is_in(znormalizovani_top_zahranicni))), title=alt.TitleParams("…a překladovou literaturou", subtitle="Zahraniční autoři vybraní podle stejného klíče, bez vydání mimo ČR."), width=300).mark_circle(size=8) \
#       .encode(
#            x=alt.X("rok:T", title=None, axis=alt.Axis(domainOpacity=0, tickColor='#DCDDD6')), 
#            y=alt.Y("jmeno:N", sort=znormalizovani_top_zahranicni_razeni, title=None, axis=alt.Axis(orient='left', domainOpacity=0, tickColor='#DCDDD6')), 
#            yOffset=alt.YOffset("jitter:Q", scale=alt.Scale(range=[3, 15])), 
#            color=alt.Color('jmeno:N', scale=alt.Scale(range=['#D6534B']), #, '#445B78', '#DB842F', '#70871E']), 
#                            sort=znormalizovani_top_zahranicni_razeni).legend(None)) \
#        .transform_calculate(jitter="sqrt(-2*log(random()))*cos(2*PI*random())")

# alt.vconcat(norm_dom, norm_zahr).configure_view(stroke='transparent').resolve_scale(color='independent',x="shared")

norm_dom.configure_view(stroke='transparent').resolve_scale(color='independent',x="shared")

alt.Chart(...)

In [721]:
dografu

jmeno,titul,rok,umrti,kde
str,str,i64,i64,str
"""Karel Mácha""","""Revoluční humanismus St.K. Neu…",1954,null,"""domaci"""
"""Stanislav Sohr""","""Breviář""",1995,2000,"""domaci"""
"""Eva Kantůrková""","""Památník""",1995,null,"""domaci"""
"""Arnošt Lustig""","""Hořká vůně mandlí""",1968,2011,"""domaci"""
"""Václav Havel""","""Protokoly""",1966,2011,"""domaci"""
"""Jan Procházka""","""Rok života""",1956,1971,"""domaci"""
"""Anna Sedlmayerová""","""Čtyři detektivky""",1970,1995,"""domaci"""
"""Vladimír Škutina""","""Rodič v síti""",1991,1995,"""domaci"""
"""Josef Škvorecký""","""Podivný pán z Providence a jin…",1999,2012,"""domaci"""


In [823]:
# First create the base chart parameters
base = alt.Chart(alt_friendly(dografu), 
    title=alt.TitleParams(
        ["Normalizační průrva českou literaturou"], 
        subtitle=[f"{kolik} autorů a autorek žijících v době srpnové okupace, kterým toho vyšlo nejvíc",
                 "v druhé půlce 60. let a první půlce 90. let a zároveň nic mezi lety 1973 a 1987.",
                 "Modré tečky jsou knihy vydané v zahraničí, tlusté čárky jsou rok úmrtí."]
    ), width=300)

# Create the yellow dots layer (foreign publications)
foreign = base.mark_circle().encode(
    x=alt.X("rok:T", title=None, axis=alt.Axis(domainOpacity=0, tickColor='#DCDDD6')),
    y=alt.Y("jmeno:N", sort=znormalizovani_top_domaci_razeni, title=None, 
            axis=alt.Axis(orient='left', domainOpacity=0, tickColor='#DCDDD6')),
    yOffset=alt.YOffset("jitter:Q", scale=alt.Scale(range=[3, 15])),
    size=alt.value(5),
    color=alt.value('#81A9D5')
).transform_filter(
    alt.datum.kde == 'zahranici'
).transform_calculate(
    jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
)

# Create the domestic publications layer
domestic = base.mark_circle().encode(
    x=alt.X("rok:T", title=None, axis=alt.Axis(domainOpacity=0, tickColor='#DCDDD6')),
    y=alt.Y("jmeno:N", sort=znormalizovani_top_domaci_razeni, title=None, 
            axis=alt.Axis(orient='left', domainOpacity=0, tickColor='#DCDDD6')),
    yOffset=alt.YOffset("jitter:Q", scale=alt.Scale(range=[3, 15])),
    size=alt.value(8),
    color=alt.Color('domaci:N', 
                   scale=alt.Scale(range=['#D6534B']), 
                   sort=znormalizovani_top_domaci_razeni,
                   legend=None)
).transform_filter(
    alt.datum.kde == 'domaci'
).transform_calculate(
    jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
)

kdy_umreli = base.mark_tick(
    color='black',  # optional: you can specify color
    thickness=1.75,
    height=9
).encode(
    x=alt.X('umrti:T', title=None),
    y=alt.Y("jmeno:N", sort=znormalizovani_top_domaci_razeni, title=None, 
            axis=alt.Axis(orient='left', domainOpacity=0, tickColor='#DCDDD6'))
)

# Layer the charts with foreign (yellow) on bottom and domestic on top
norm_dom = alt.layer(foreign, domestic, kdy_umreli).configure_view(stroke='transparent').resolve_scale(color='independent',x="shared")

norm_dom

alt.LayerChart(...)

In [850]:
norm_dom.save("grafy/odmlky.svg")

In [755]:
kdy_umreli

alt.Chart(...)

In [288]:
znormalizovani_top_zahranicni_razeni

['Erich von Däniken',
 'Ladislav Mňačko',
 'Peter Ferdinand Drucker',
 'Simone de Beauvoir',
 'Jean-Paul Sartre',
 'Erich Fromm',
 'Rochdi Goulli',
 'Max Frisch',
 'Jean Genet',
 'Ladislav Grosman',
 'Eugène Ionesco',
 'Clifford D. Simak',
 'Peter Weiss',
 'Richmal Crompton',
 'Poul Anderson',
 'Marguerite Duras',
 'Günter Grass',
 'John Knittel',
 'Jan Blahoslav Čapek',
 'Nathalie Sarraute',
 'Karl Jaspers',
 'Michel Quoist',
 'Cornelius Ryan',
 'Gertrud von Le Fort',
 'Wolfgang Ott']

## Stopky

In [158]:
nechceme = ['jn19990210621']
df = df.filter(~pl.col('100_7').is_in(nechceme))

In [159]:
def nevysli_po(rok, obdobi, nasobek):
    vysli_pred = df.filter(pl.col("rok").is_between(rok-obdobi,rok)).group_by('100_7').len().filter(pl.col('len') > 3).select(pl.col("100_7")).to_series().to_list()
    vysli_po = set(df.filter(pl.col("rok") > rok).select(pl.col("100_7")).to_series().to_list())
    ukonceni = [v for v in vysli_pred if v not in vysli_po]
    print(f"Filtru vyhovuje {len(ukonceni)} lidí.")
    return df.filter(pl.col("100_7").is_in(ukonceni)).filter(pl.col("rok").is_between(rok-(obdobi * nasobek),rok)).group_by(["jmeno","100_7"]).len().sort(by="len",descending=True).head(100)

In [320]:
sedesatosm = nevysli_po(1993, 2, 20).head(3)
sedesatosm

Filtru vyhovuje 75 lidí.


jmeno,100_7,len
str,str,u32
"""Jiří Kašpar""","""jk01053180""",34
"""Josef Kubalík""","""jk01070249""",31
"""Vladimír Klevis""","""jk01060437""",30


In [322]:
tricetosm = nevysli_po(1938, 2, 20).head(3)
tricetosm

Filtru vyhovuje 78 lidí.


jmeno,100_7,len
str,str,u32
"""Josef Skružný""","""jk01111617""",114
"""Augustin Matolín""","""jk01080750""",97
"""Richard Fischer""","""jk01031292""",72


In [324]:
ctyricetosm = nevysli_po(1948, 2, 20).head(3)
ctyricetosm

Filtru vyhovuje 145 lidí.


jmeno,100_7,len
str,str,u32
"""Quido Maria Vyskočil""","""jk01151235""",149
"""František Pulec""","""jk01101720""",75
"""Josef Křenek""","""jk01070044""",72


In [326]:
osmnact = nevysli_po(1918, 2, 20).head(3)
osmnact

Filtru vyhovuje 8 lidí.


jmeno,100_7,len
str,str,u32
"""Josef Flekáček""","""jk01031426""",26
"""Václav Simandl""","""jk01111320""",10
"""Josef Bubeníček""","""jk01020080""",9


In [328]:
ctyricetosm = nevysli_po(1948, 2, 20).head(3)
ctyricetosm

Filtru vyhovuje 145 lidí.


jmeno,100_7,len
str,str,u32
"""Quido Maria Vyskočil""","""jk01151235""",149
"""František Pulec""","""jk01101720""",75
"""Josef Křenek""","""jk01070044""",72


In [330]:
nevysli_po(1968, 2, 20).head(3)

Filtru vyhovuje 24 lidí.


jmeno,100_7,len
str,str,u32
"""Antonín Václavovič""","""jk01141051""",24
"""Miluše Nohejlová""","""jx20040805028""",23
"""Jitka Voříšková""","""jx20040721068""",22


In [332]:
osmdesatdevet = nevysli_po(1989, 2, 20).head(3)
osmdesatdevet

Filtru vyhovuje 81 lidí.


jmeno,100_7,len
str,str,u32
"""František Buriánek""","""jk01020403""",68
"""Marta Chytilová""","""jk01050494""",62
"""František Egermayer""","""jk01030317""",61


In [167]:
df = df.with_columns(pl.col("jmeno").replace({"Ladislav Riedl Německobrodský": "L. R. Německobrodský"}))

In [334]:
#fertig = pl.concat([osmnact, ctyricetosm, osmdesatdevet])

fertig_18_kdo = osmnact.select(pl.col('100_7')).to_series().to_list()
fertig_18_razeni = osmnact.select(pl.col('jmeno')).to_series().to_list()
fertig_18_do_grafu = df.filter(pl.col('100_7').is_in(fertig_18_kdo))

fertig_48_kdo = ctyricetosm.select(pl.col('100_7')).to_series().to_list()
fertig_48_razeni = ctyricetosm.select(pl.col('jmeno')).to_series().to_list()
fertig_48_do_grafu = df.filter(pl.col('100_7').is_in(fertig_48_kdo))

fertig_89_kdo = osmdesatdevet.select(pl.col('100_7')).to_series().to_list()
fertig_89_razeni = osmdesatdevet.select(pl.col('jmeno')).to_series().to_list()
fertig_89_do_grafu = df.filter(pl.col('100_7').is_in(fertig_89_kdo))

In [169]:
fertig_48_razeni

['Quido Maria Vyskočil', 'Ladislav Riedl Německobrodský', 'František Pulec']

In [336]:
fertig_18_graf = alt.Chart(alt_friendly(fertig_18_do_grafu.filter(pl.col('rok').is_between(1900,2000))), title=alt.TitleParams(["Čeští autoři, kteří přestali vycházet","po vyhlášení Československa…"], 
)).mark_circle(size=8) \
        .encode(
            x=alt.X("rok:T", title=None, axis=alt.Axis(domainOpacity=0, tickColor='#DCDDD6'), scale=alt.Scale(domain=[{'year': 1900},{'year':2000}])), 
            y=alt.Y("jmeno:N", sort=fertig_18_razeni, title=None, axis=alt.Axis(orient='left', domainOpacity=0, tickColor='#DCDDD6')), 
            yOffset=alt.YOffset("jitter:Q", scale=alt.Scale(range=[3, 15])), 
            color=alt.Color('jmeno:N', scale=alt.Scale(range=['#D6534B', '#445B78', '#DB842F', '#70871E']), 
                            sort=fertig_18_razeni).legend(None)).transform_calculate(jitter="sqrt(-2*log(random()))*cos(2*PI*random())")

fertig_48_graf = alt.Chart(alt_friendly(fertig_48_do_grafu.filter(pl.col('rok').is_between(1900,2000))), title=alt.TitleParams(["…po únorovém převratu…"], 
)).mark_circle(size=8) \
        .encode(
            x=alt.X("rok:T", title=None, axis=alt.Axis(domainOpacity=0, tickColor='#DCDDD6'), scale=alt.Scale(domain=[{'year': 1900},{'year':2000}])), 
            y=alt.Y("jmeno:N", sort=fertig_48_razeni, title=None, axis=alt.Axis(orient='left', domainOpacity=0, tickColor='#DCDDD6')), 
            yOffset=alt.YOffset("jitter:Q", scale=alt.Scale(range=[3, 15])), 
            color=alt.Color('jmeno:N', scale=alt.Scale(range=['#D6534B', '#445B78', '#DB842F', '#70871E']), 
                            sort=fertig_48_razeni).legend(None)).transform_calculate(jitter="sqrt(-2*log(random()))*cos(2*PI*random())")

fertig_89_graf = alt.Chart(alt_friendly(fertig_89_do_grafu.filter(pl.col('rok').is_between(1900,2000))), title=alt.TitleParams(["…a po revoluci"], 
)).mark_circle(size=8) \
        .encode(
            x=alt.X("rok:T", title=None, axis=alt.Axis(domainOpacity=0, tickColor='#DCDDD6'), scale=alt.Scale(domain=[{'year': 1900},{'year':2000}])), 
            y=alt.Y("jmeno:N", sort=fertig_89_razeni, title=None, axis=alt.Axis(orient='left', domainOpacity=0, tickColor='#DCDDD6')), 
            yOffset=alt.YOffset("jitter:Q", scale=alt.Scale(range=[3, 15])), 
            color=alt.Color('jmeno:N', scale=alt.Scale(range=['#D6534B', '#445B78', '#DB842F', '#70871E']), 
                            sort=fertig_89_razeni).legend(None)).transform_calculate(jitter="sqrt(-2*log(random()))*cos(2*PI*random())")

alt.vconcat(fertig_18_graf, fertig_48_graf, fertig_89_graf).resolve_axis(x="shared").configure_view(stroke='transparent')

alt.VConcatChart(...)